# EMS ROUTING

In [1]:
import folium
import json
import numpy as np

from h3 import h3
from folium.plugins import MarkerCluster

In [2]:
tile = 'https://api.mapbox.com/styles/v1/mapbox/streets-v10/tiles/256/{z}/{x}/{y}?access_token=pk.eyJ1IjoiZGl2OSIsImEiOiJjam9jZWo5eGYwbW0zM3BydHNnbm5wbGs5In0.VRTb28kY6k_5WObVO6-QZg'
m = folium.Map(location=[42.4485, -76.4804],
               zoom_start=13,
               detect_retina=True)

m.add_tile_layer(tiles=tile,
                 attr='Map data &copy; <a href="http://openstreetmap.org">OpenStreetMap</a> contributors, ' +
                      '<a href="http://creativecommons.org/licenses/by-sa/2.0/">CC-BY-SA</a>, ' +
                      'Imagery © <a href="http://mapbox.com">Mapbox</a>',
                 name="Mapbox Street",
                 detect_retina=True)
# 
style = lambda x: {'fillColor': '#100000', 'fillOpacity':0}
 
with open('ithaca.json','r') as f:
    geoJson = json.load(f)
    
geo = folium.GeoJson(
    geoJson,
    name='boundary',
    style_function=style
).add_to(m)

In [3]:
# Resolution goes from 0..15, with 15 being the max
res = 10

hexagons = list(h3.polyfill(geoJson,res, True))
edge = h3.edge_length(res) * 1e3

print("Num Hexagons:", len(hexagons))
print("Edge Length (m):", edge)

Num Hexagons: 1000
Edge Length (m): 65.907807


In [4]:
def grid_style(feature):
    return {
        'fillOpacity': 0.3,
        'color': 'red'
    }

# Get hexagon coordinates
coords = list(map(lambda x: [h3.h3_to_geo_boundary(x, True)], hexagons))
gridJson = {}
geom = {}
geom["type"] = "MultiPolygon"
geom["coordinates"] = coords

gridJson["type"] = "Feature"
gridJson["geometry"] = geom

# Plot Grid
folium.GeoJson(
    gridJson,
    name='grid',
    style_function=grid_style
    ).add_to(m)

# Add Parking Markers
park_spots = np.loadtxt('parking.txt')

#Create a Folium feature group for this layer
pt_lyr = folium.FeatureGroup(name = 'parking spots')
icons=[folium.Icon(icon='product-hunt', prefix='fa') for i in range(len(park_spots))]
#Add the clustered points of parking spots to this this layer
pt_lyr.add_child(MarkerCluster(locations = park_spots, icons=icons))
m.add_child(pt_lyr)

folium.LayerControl().add_to(m)

m.save('routing.html')
m